In [13]:
import os
import warnings
import requests
import json

from openai import OpenAI
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage, UpstageLayoutAnalysisLoader
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_chroma import Chroma
from tavily import TavilyClient

warnings.filterwarnings("ignore")

%load_ext dotenv
%dotenv

client = OpenAI(
    api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
)
llm = ChatUpstage()
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [156]:
template = """
<<system>>
신조어란 *새로 만들어진* 단어 및 용어 가운데 표준어로 등재되지 않은 말을 뜻합니다.
당신은 신조어를 매우 잘 알고 있는 전문가로, [SENTENCE]에서 신조어 단어를 찾아 나열하고 있습니다.
신조어를 나열하는 것 이외에는 절대 대답하지 않습니다.
<</system>>
---
[SENTENCE]
{sentence}
[/SENTENCE]
"""

prompt_template = PromptTemplate.from_template(template)
chain = prompt_template | llm | StrOutputParser()

In [157]:
sentence = """우리 언니가 말없이 폰 붙들고 그딴 시답지 않은 개소리나 듣고 있길래 킹받아서 쫓아왔어요. 마누라 친구랑 하룻밤 재미봐놓고 들통나니 어쩔티비 내배째라?
언닌 그쪽 식구들한테 별의별 짓 다 당하면서 견디고 있는데, 이렇게 애매한 스탠스가 얼마나 사람 마음 힘들게 하는지 아세요?
욕 먹는 게 겁나요? 욕 한 바가지 드신다고 안 죽습니다!
사랑은 적어도 미안하단 말 하지 않는 게 기본 상식이에요. 이딴 것도 사랑이라고. 할많하않입니다 진짜."""

In [158]:
new_list = chain.invoke({"sentence": sentence}).split(", ")

In [168]:
context = []
for word in new_list:
    context_eng = tavily.search(query="What is meaing " + word, search_depth="advanced")['results']
    context_kor = tavily.search(query=word + " 무슨 뜻", search_depth="advanced")['results']
    context += context_eng + context_kor

print(context)

[{'title': '킹받다 - Wiktionary, the free dictionary', 'url': 'https://en.wiktionary.org/wiki/킹받다', 'content': '킹받아서 kingbadaseo: 킹받으니 kingbadeuni: 킹받으니까 kingbadeunikka: Contrast 킹받지만 kingbatjiman: 킹받는데 kingbanneunde: 킹받더니 kingbatdeoni: Conjunction 킹받고 kingbatgo: Condition 킹받으면 kingbadeumyeon: 킹받아야 kingbadaya: Motive 킹받으려고 kingbadeuryeogo: Noun and determiner forms; Verbal ...', 'score': 0.97378, 'raw_content': None}, {'title': 'Google Translate', 'url': 'https://translate.google.com.pk/', 'content': "Google's service, offered free of charge, instantly translates words, phrases, and web pages between English and over 100 other languages.", 'score': 0.96531, 'raw_content': None}, {'title': 'What is the meaning of "킹받아"? - Question about Korean | HiNative', 'url': 'https://hinative.com/questions/25371037', 'content': '킹받아 (king-bad-a) Definition of 킹받아 킹받아=열받아=I\'m pissed off "열 heat"를 "킹 king"으로 바꿔쓰는 밈이 있어유 유행한지 몇 달 됐는데 지금도 많이 쓰여유 ... 너무 킹받아서 그런 말을 했어. (Aku sangat marah jadi berkata sepert